In [1]:
# Importing all the essential libraries into jupyter notebook 

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from tensorflow.keras.metrics import Accuracy, Precision, Recall

In [2]:
# Building parameters for the images
    
image_rows, image_columns = 150, 150
num_classes = 2
batch_size = 32

# Setting the directories for cats and dogs datasets

Dogs_dir = '/Users/user/Desktop/ML_Project/PetImages/Dog'
Cats_dir = '/Users/user/Desktop/ML_Project/PetImages/Cat'

In [3]:
# Analising Dog images

Dogs_images = []
m = 0
for n in os.listdir(Dogs_dir):
    image = cv2.imread(os.path.join(Dogs_dir, n), cv2.IMREAD_GRAYSCALE)
    if image is not None:
        Dogs_images.append(image)
        m += 1;
        if m >1000:
            break;

# Analising cat images

Cats_images = []
m = 0;
for n in os.listdir(Cats_dir):
    image = cv2.imread(os.path.join(Cats_dir, n), cv2.IMREAD_GRAYSCALE)
    if image is not None:
        Cats_images.append(image)
        m +=1;
        if m >1000:
            break;

In [4]:
# Setting the proportion of images to same size

Dogs_images = [cv2.resize(n, (image_rows, image_columns)) for n in Dogs_images]
Cats_images = [cv2.resize(n, (image_rows, image_columns)) for n in Cats_images]

# Translating images to arrays

Dogs_images = np.array(Dogs_images)
Cats_images = np.array(Cats_images)

In [5]:
# Designing labels and combining two images sets

X = np.concatenate((Dogs_images, Cats_images), axis=0)
Y = np.concatenate((np.zeros(len(Dogs_images)), np.ones(len(Cats_images))), axis=0)

# Breaking train and test data sets to split

X_train, X_test, Y_train,Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [6]:
# By normalizing enchanting the data and encoding the labels

X_train = X_train.reshape(X_train.shape[0], image_rows, image_columns, 1)
X_test = X_test.reshape(X_test.shape[0], image_rows, image_columns, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_train = np_utils.to_categorical(Y_train, num_classes)
Y_test = np_utils.to_categorical(Y_test, num_classes)

In [7]:
# Designing CNN deep learning model with all the layers

model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# Arranging the model
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy', tf.keras.metrics.Precision() ])


In [8]:
# data accretion
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


In [9]:
# Arranging the model on the accretion dataset

datagen.fit(X_train)
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
steps_per_epoch=len(X_train) // batch_size,
epochs=20,
validation_data=(X_test, Y_test),
validation_steps=len(X_test) // batch_size)

# On the test data set estimating the model

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test data loss:', score[0])
print('Test data Accuracy:', score[1])
print('Test data Precision:', score[2])

C:\Users\user\AppData\Local\Temp\ipykernel_19860\3804672967.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),


Epoch 1/20
43/43 [==============================] - 63s 1s/step - loss: 1.5258 - accuracy: 0.5296 - precision: 0.5296 - val_loss: 0.6923 - val_accuracy: 0.4892 - val_precision: 0.4892
Epoch 2/20
43/43 [==============================] - 60s 1s/step - loss: 0.7458 - accuracy: 0.5201 - precision: 0.5201 - val_loss: 0.6868 - val_accuracy: 0.4908 - val_precision: 0.4908
Epoch 3/20
43/43 [==============================] - 60s 1s/step - loss: 0.7090 - accuracy: 0.5157 - precision: 0.5157 - val_loss: 0.6848 - val_accuracy: 0.5125 - val_precision: 0.5125
Epoch 4/20
43/43 [==============================] - 60s 1s/step - loss: 0.6914 - accuracy: 0.5245 - precision: 0.5245 - val_loss: 0.6851 - val_accuracy: 0.5208 - val_precision: 0.5208
Epoch 5/20
43/43 [==============================] - 60s 1s/step - loss: 0.6850 - accuracy: 0.5566 - precision: 0.5566 - val_loss: 1.1596 - val_accuracy: 0.5092 - val_precision: 0.5092
Epoch 6/20
43/43 [==============================] - 60s 1s/step - loss: 0.7069 -

In [10]:
# Comparing our deep learning model with the Decision Tree, KNN and, SVM Classifier machine learning models

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.svm import SVC

In [11]:
#fixing axis

Y_train = Y_train.argmax(axis=1)
Y_test = Y_test.argmax(axis=1)

In [12]:
# Decision tree, KNN, SVM Classifier models 

DT_clf = DecisionTreeClassifier()
KNN_clf = KNeighborsClassifier()
SVM_clf = SVC()


In [13]:
# Installing the models on train dataset

DT_clf.fit(X_train.reshape(X_train.shape[0], image_rows*image_columns), Y_train)
KNN_clf.fit(X_train.reshape(X_train.shape[0], image_rows*image_columns), Y_train)
SVM_clf.fit(X_train.reshape(X_train.shape[0], image_rows*image_columns), Y_train)


SVC()

In [15]:
# Valuate the models on the test data

DT_preds = DT_clf.predict(X_test.reshape(X_test.shape[0], image_rows*image_columns))
KNN_preds = KNN_clf.predict(X_test.reshape(X_test.shape[0], image_rows*image_columns))
SVM_preds = SVM_clf.predict(X_test.reshape(X_test.shape[0], image_rows*image_columns))


In [17]:
# Evaluating the accuracies of all the traditional machine learning models

DT_accuracy = accuracy_score(Y_test, DT_preds)
KNN_accuracy = accuracy_score(Y_test, KNN_preds)
SVM_accuracy = accuracy_score(Y_test, SVM_preds)

In [18]:
# Evaluating the precision of all the traditional machine learning models

DT_precision = precision_score(Y_test, DT_preds, average='weighted')
KNN_precision = precision_score(Y_test, KNN_preds, average='weighted')
SVM_precision = precision_score(Y_test, SVM_preds, average='weighted')

In [19]:
# Evaluating the loss of all the traditional machine learning models

DT_loss = DT_clf.score(X_test.reshape(X_test.shape[0], image_rows*image_columns), Y_test)
KNN_loss = KNN_clf.score(X_test.reshape(X_test.shape[0], image_rows*image_columns), Y_test)
SVM_loss = SVM_clf.score(X_test.reshape(X_test.shape[0], image_rows*image_columns), Y_test)

In [21]:
# Comparing CNN model with all other traditional models

print(' Comparing the performance of deep learning model with Decision tree, KNN and, SVM machine learning models')
print(f'The accuracy of CNN model: {score[1]} vs Decision Tree: {DT_accuracy} vs KNN: {KNN_accuracy} vs SVM: {SVM_accuracy}')
print(f'The Precision of CNN model: {score[2]} vs Decision Tree: {DT_precision} vs KNN: {KNN_precision} vs SVM: {SVM_precision}')
print(f'The Loss of CNN model: {score[0]} vs Decision Tree: {DT_loss} vs KNN: {KNN_loss} vs SVM: {SVM_loss}')


 

 Comparing the performance of deep learning model with Decision tree, KNN and, SVM machine learning models
The accuracy of CNN model: 0.6871880292892456 vs Decision Tree: 0.5474209650582362 vs KNN: 0.5640599001663894 vs SVM: 0.5873544093178037
The Precision of CNN model: 0.6871880292892456 vs Decision Tree: 0.5483944332914681 vs KNN: 0.5658030443776083 vs SVM: 0.5880780305373614
The Loss of CNN model: 0.598701536655426 vs Decision Tree: 0.5474209650582362 vs KNN: 0.5640599001663894 vs SVM: 0.5873544093178037
